In [1]:
from util import *
import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn
import sklearn.feature_extraction.text as text

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import decomposition

In [2]:
lines=[]
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  

In [3]:
userid = []
rating = []
docs = []
business = []
i = 0
j = 0
error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        docs.append(l[3])
        #d = clean(l[3])
        #kmers = getKmers(d)
        #d.extend(kmers)
        #docs.append(d)
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)



Training Data: Number of lines processed: 317586
Training Data: Length of userid array: 317586
Training Data: Length of rating array: 317586
Training Data: Length of docs array: 317586
Training Data: Length of business array: 317586
Training Data: Number of exceptions encountered: 0


In [4]:

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 50000
n_features = 15000
n_components = 20
n_top_words = 50

data_samples = docs[:n_samples]

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

tf_lda= lda.transform(tf)


print("\nTopics in LDA model:")#tfidf_feature_names = tfidf_vectorizer.get_feature_names()
vocab = np.array(tf_vectorizer.get_feature_names())
#print_top_words(nmf, tfidf_feature_names, n_top_words)
#print_top_words(lda, tf_feature_names, n_top_words)



Extracting tf features for LDA...
done in 9.747s.
()
Fitting LDA models with tf features, n_samples=50000 and n_features=15000...
done in 125.964s.

Topics in LDA model:


In [5]:
trainTopics= tf_lda
trainTopics = trainTopics / np.sum(trainTopics, axis=1, keepdims=True)


In [6]:
print(trainTopics)

[[ 0.00147059  0.00147059  0.03919865 ...,  0.00147059  0.14004899
   0.00147059]
 [ 0.000625    0.000625    0.56523961 ...,  0.000625    0.000625    0.000625  ]
 [ 0.00058824  0.00058824  0.00058824 ...,  0.00058824  0.00058824
   0.25043321]
 ..., 
 [ 0.00060241  0.07531244  0.00060241 ...,  0.09287984  0.00060241
   0.00060241]
 [ 0.00069444  0.05699992  0.00069444 ...,  0.00069444  0.01781645
   0.01815238]
 [ 0.00069444  0.00069444  0.00069444 ...,  0.00069444  0.00069444
   0.00069444]]


In [7]:
d, f = trainTopics.shape
cols = ["Topic"+str(i) for i in xrange(1, f+1)]
nmfDF = pd.DataFrame(trainTopics, columns=cols)

In [8]:
nmfDF['rating'] = map(float,rating[:50000])

In [9]:
nmfDF.T.head(25)

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
Topic1,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.218595,0.040985,...,0.002381,0.003333,0.002778,0.001000,0.025883,0.001136,0.002381,0.000602,0.000694,0.000694
Topic2,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.066922,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.001136,0.002381,0.075312,0.057000,0.000694
Topic3,0.039199,0.565240,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.001136,0.002381,0.000602,0.000694,0.000694
Topic4,0.139249,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.082268,0.000485,0.059247,0.087028,0.000602,0.000694,0.000694
Topic5,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.001136,0.002381,0.000602,0.020279,0.074946
Topic6,0.186654,0.000625,0.280744,0.919819,0.371056,0.001087,0.003333,0.614252,0.491485,0.002500,...,0.002381,0.003333,0.002778,0.387107,0.381562,0.101366,0.002381,0.643681,0.036944,0.123146
Topic7,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.061146,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.044745,0.002381,0.000602,0.023663,0.000694
Topic8,0.228225,0.292700,0.221520,0.040051,0.002083,0.402953,0.003333,0.002500,0.001163,0.369296,...,0.487511,0.470501,0.002778,0.338489,0.115352,0.515464,0.002381,0.045142,0.401892,0.074176
Topic9,0.246036,0.042084,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.058243,0.002500,...,0.002381,0.003333,0.093405,0.036717,0.014531,0.001136,0.002381,0.000602,0.185869,0.000694
Topic10,0.001471,0.000625,0.000588,0.000633,0.002083,0.152448,0.003333,0.215221,0.108480,0.002500,...,0.002381,0.003333,0.856595,0.066902,0.000485,0.001136,0.002381,0.000602,0.000694,0.000694


In [10]:
def getSentiment(x):
    if x < 3.5:
        return 0
    else:
        return 1

In [11]:
nmfDF['Sentiment'] = nmfDF['rating'].map(getSentiment)

In [12]:
nmfDF = nmfDF.dropna(how='any')


In [13]:
nmfDF.T.head(25)

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
Topic1,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.218595,0.040985,...,0.002381,0.003333,0.002778,0.001000,0.025883,0.001136,0.002381,0.000602,0.000694,0.000694
Topic2,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.066922,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.001136,0.002381,0.075312,0.057000,0.000694
Topic3,0.039199,0.565240,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.001136,0.002381,0.000602,0.000694,0.000694
Topic4,0.139249,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.082268,0.000485,0.059247,0.087028,0.000602,0.000694,0.000694
Topic5,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.001136,0.002381,0.000602,0.020279,0.074946
Topic6,0.186654,0.000625,0.280744,0.919819,0.371056,0.001087,0.003333,0.614252,0.491485,0.002500,...,0.002381,0.003333,0.002778,0.387107,0.381562,0.101366,0.002381,0.643681,0.036944,0.123146
Topic7,0.001471,0.000625,0.000588,0.000633,0.002083,0.001087,0.003333,0.061146,0.001163,0.002500,...,0.002381,0.003333,0.002778,0.001000,0.000485,0.044745,0.002381,0.000602,0.023663,0.000694
Topic8,0.228225,0.292700,0.221520,0.040051,0.002083,0.402953,0.003333,0.002500,0.001163,0.369296,...,0.487511,0.470501,0.002778,0.338489,0.115352,0.515464,0.002381,0.045142,0.401892,0.074176
Topic9,0.246036,0.042084,0.000588,0.000633,0.002083,0.001087,0.003333,0.002500,0.058243,0.002500,...,0.002381,0.003333,0.093405,0.036717,0.014531,0.001136,0.002381,0.000602,0.185869,0.000694
Topic10,0.001471,0.000625,0.000588,0.000633,0.002083,0.152448,0.003333,0.215221,0.108480,0.002500,...,0.002381,0.003333,0.856595,0.066902,0.000485,0.001136,0.002381,0.000602,0.000694,0.000694


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsRegressor

In [15]:
cols = [u'Topic1', u'Topic2', u'Topic3', u'Topic4', u'Topic5', u'Topic6', u'Topic7', u'Topic8', u'Topic9', u'Topic10', u'Topic11', u'Topic12', u'Topic13', u'Topic14', u'Topic15', u'Topic16', u'Topic17', u'Topic18', u'Topic19', u'Topic20', u'Sentiment']
Xtrain = nmfDF[:40000][cols]
Ytrain = nmfDF[:40000]['rating']
Xtest = nmfDF[40000:][cols]
Ytest = nmfDF[40000:]['rating']


#Xtrain_base = trainVectors[:18000]
#Ytrain_base = map(float,rating[:18000])
#Xtest_base = trainVectors[8000:10000]
#Ytest_base = map(float,rating[8000:10000])




In [16]:
from sklearn.metrics import mean_squared_error, r2_score
clfs = [ LogisticRegression(),KNeighborsRegressor(n_neighbors=3)]
clf_names = ['Logistic Regression','KNeighborsRegressor']
results = {}
for (i, clf_) in enumerate(clfs):
    clf = clf_.fit(Xtrain, Ytrain)
    preds = clf.predict(Xtest)
    
    #clf1 = clf_.fit(Xtrain_base, Ytrain_base)
    #preds1 = clf.predict(Xtest_base)
   
    
 
    #precision = metrics.precision_score(Ytest, preds)
    #recall = metrics.recall_score(Ytest, preds)
    #f1 = metrics.f1_score(Ytest, preds)
    #accuracy = accuracy_score(Ytest, preds)
    #report = classification_report(Ytest, preds)
    #matrix = metrics.confusion_matrix(Ytest, preds, labels=[1, 2, 3, 4, 5])
    score= r2_score(Ytest, preds)
    #score1= r2_score(Ytest_base, preds1)
    print "NMF: " +clf_names[i] + str(score)
    #print "Baseline : " +clf_names[i] + str(score)



NMF: Logistic Regression0.550523995324
NMF: KNeighborsRegressor0.643909588418


In [ ]:
print results